# Build Dataset: MagangHub Vacancies (Full Pipeline)

Notebook ini menjalankan pipeline lengkap:
1. **Fetch** data lowongan dari API MagangHub
2. **Prepare** (flatten, parse JSON, enrich skills)
3. **Score** setiap lowongan berdasarkan peluang
4. Menyimpan dataset siap analisis (`vacancies_scored.parquet`)

> Direktori hasil:  
> - Raw JSON → `data/raw/run_*`  
> - Clean parquet → `data/clean/vacancies.parquet`  
> - Scored parquet → `data/clean/vacancies_scored.parquet`


In [3]:
import os
import sys
from pathlib import Path

import pandas as pd
from IPython.display import display, Markdown

# Pastikan path ke root project
ROOT = Path(/C/Users/Azahr/OneDrive/Dokumen/kerja/Ngoding/magang-intel).resolve().parents[1]
sys.path.append(str(ROOT / "src"))

from utils import load_yaml, ensure_dir, preview, Timer


SyntaxError: invalid syntax (3813088999.py, line 9)

In [6]:
import sys
from pathlib import Path
from IPython.display import display, Markdown

HERE = Path.cwd() 
ROOT = None
for c in [HERE, HERE.parent, HERE.parent.parent]:
    if (c / "src").exists() and (c / "data").exists():
        ROOT = c
        break

if ROOT is None:
    if (HERE / "src").exists() and (HERE / "data").exists():
        ROOT = HERE
    else:
        raise FileNotFoundError("Tidak dapat menemukan folder root proyek ('src' & 'data').")

if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))

display(Markdown(f"**Project Root:** `{ROOT}`"))
display(Markdown(f"**Sys.path updated:** `{str(ROOT)}` ditambahkan."))

from src import fetch 
from src.utils import Timer 
print("Semua modul berhasil di-import.")

**Project Root:** `c:\Users\Azahr\OneDrive\Dokumen\kerja\Ngoding\magang-intel`

**Sys.path updated:** `c:\Users\Azahr\OneDrive\Dokumen\kerja\Ngoding\magang-intel` ditambahkan.

Semua modul berhasil di-import.


## 🔹 Step 1 — Fetch Data
Menjalankan `src/fetch.py` untuk menarik semua lowongan MagangHub dan menyimpan JSON mentah di `data/raw/run_*`.

In [7]:
from src import fetch
timer = Timer("Fetch Data")

fetch.main()  # otomatis ambil semua halaman (pages=0/"all" di params.yaml)
timer.stop()


[INFO] Page 1 OK | items=100 | total=24708 | last_page=248 | per_page=100
[INFO] Page 2 OK | items=100
[INFO] Page 3 OK | items=100
[INFO] Page 4 OK | items=100
[INFO] Page 5 OK | items=100
[INFO] Page 6 OK | items=100
[INFO] Page 7 OK | items=100
[INFO] Page 8 OK | items=100
[INFO] Page 9 OK | items=100
[INFO] Page 10 OK | items=100
[INFO] Page 11 OK | items=100
[INFO] Page 12 OK | items=100
[INFO] Page 13 OK | items=100
[INFO] Page 14 OK | items=100
[INFO] Page 15 OK | items=100
[INFO] Page 16 OK | items=100
[INFO] Page 17 OK | items=100
[INFO] Page 18 OK | items=100
[INFO] Page 19 OK | items=100
[INFO] Page 20 OK | items=100
[DONE] Saved to C:\Users\Azahr\OneDrive\Dokumen\kerja\Ngoding\magang-intel\data\raw\run_20251111_120236_2a69155a
[TIMER] Fetch Data: 118.80s


118.802901

## 🔹 Step 2 — Prepare Dataset
Gabungkan seluruh file JSON dari `data/raw/`, flatten kolom penting, parse jadwal & jenjang,
hitung kolom turunan (`competition_ratio`, `days_to_deadline`),
serta ekstraksi skill dengan `config/skills.yaml`.

In [ ]:
from src import prepare

timer = Timer("Prepare Data")
prepare.main()
timer.stop()

clean_path = ROOT / "data" / "clean" / "vacancies.parquet"
df_clean = pd.read_parquet(clean_path)
preview(df_clean)

## 🔹 Step 3 — Hitung Skor & Ranking
Gunakan bobot dari `config/weights.yaml` untuk menghasilkan kolom:
`freshness_score`, `quota_score`, `competition_score`, dan `priority_score`.

In [ ]:
from src import score

timer = Timer("Score Data")
score.main()
timer.stop()

scored_path = ROOT / "data" / "clean" / "vacancies_scored.parquet"
df = pd.read_parquet(scored_path)
display(Markdown("### 🎯 Top 10 Lowongan (berdasarkan priority_score)"))
df.head(10)[["rank","posisi","nama_perusahaan","nama_provinsi","priority_score"]]

## 🔹 Step 4 — Cek Ringkasan Cepat
Lihat total baris, persentase lowongan terkait *data/analitik*, serta ringkasan skor.

In [ ]:
n_total = len(df)
n_data_related = df["is_data_related"].sum()
avg_score = df["priority_score"].mean()

display(Markdown(f"""
**Total lowongan:** {n_total:,}  
**Lowongan terkait data:** {n_data_related:,} ({n_data_related/n_total*100:.1f}%)  
**Rata-rata skor peluang:** {avg_score:.2f}
"""))

## 🔹 Step 5 — Simpan Snapshot Harian
Dataset ini akan digunakan untuk analisis tren & visualisasi.

In [ ]:
import datetime as dt

today = dt.date.today().strftime("%Y-%m-%d")
snap_path = ROOT / "output" / "tables" / f"vacancies_snapshot_{today}.csv"
ensure_dir(snap_path.parent)
df.to_csv(snap_path, index=False, encoding="utf-8-sig")
print(f"[SAVED] {snap_path}")


# ✅ Pipeline Selesai!
Dataset lengkap telah dibangun 🎉

📂 Hasil utama:
- `data/clean/vacancies.parquet`
- `data/clean/vacancies_scored.parquet`
- `output/tables/top_recommendations.xlsx`

Lanjut ke **`02_analysis.ipynb`** untuk visualisasi:
- Profesi terpadat  
- Provinsi dengan perusahaan terbanyak  
- Skill paling sering muncul  
- Peluang terbesar (rasio pelamar/kuota terendah)
